<a href="https://colab.research.google.com/github/OmarMachuca851/Task/blob/main/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow

## Problema 1: mirada retrospectiva a scratch

1. Inicialización de pesos.
2. Definir forward propagation.
3. Calcular la perdida, incluido el error medio cuadratico y cross-entropy.
4. Implementación de back propagation.
5. Implementación de la actualización de parametros de gradientes.
6. Definir número de bucle de epoca.
7. implementación de mini-batch.
8. Evaluación de precición.
9. preprocesamiento de datos: cargar, dividir y normalizar.

## Problema 2: Considerando la compatibilidad entre Scratch y TensorFlow

1. Inicialización de los pesos: `tf.Variable(tf.random_normal(...))`
2. Forward propagation: `tf.matmul`, `tf.add`, `tf.nnrelu`
3. cálculo de pérdida: `tf.nn.sigmoid_cross_entropy_with_logits`
4. Back propagation: `optimizer.minimize(loss_op)`
5. actualización de parametros: `train_op = optimizer.minimize(...)`
6. epoch loop: `for epoch in range(num_epochs)`
7. manejo con minibatch: `GetMiniBatch` con `for x, y, in ...`
8. Cálculo de accuracy: `tf.equal(...)`, `tf.reduce_mean(...)`
9. preprocesamiento de datos: `pandas`, `numpy` y `sklearn`

## Problema 3: Crear un modelo para Iris utilizando los tres tipos de variables objetivo

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Loading Iris dataset
df = pd.read_csv("data/Iris.csv")
X = df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y = df["Species"].values

# One-hot encode labels
encoder = LabelBinarizer()
y = encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Hyperparameters
lr = 0.001
batch_size = 10
num_epochs = 100
n_input = X_train.shape[1]
n_hidden1, n_hidden2 = 50, 100
n_classes = 3
n_samples = X_train.shape[0]

# Placerholder
X_ph = tf.placeholder(tf.float32, [None, n_input])
Y_ph = tf.placeholder(tf.float32, [None, n_classes])

# Weights and biases
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
    'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
    'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden1])),
    'b2': tf.Variable(tf.random_normal([n_hidden2])),
    'b3': tf.Variable(tf.random_normal([n_classes]))
}

# Model
def model(x):
    l1 = tf.nn.relu(tf.add(tf.matmul(x, weights['w1']), biases['b1']))
    l2 = tf.nn.relu(tf.add(tf.matmul(l1, weights['w2']), biases['b2']))
    return tf.matmul(l2, weights['w3'])

logits = model(X_ph)

#Loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_ph, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss_op)

# Accuracy
correct_pred = tf.equal(tf.arg_max(logits, 1), tf.argmax(Y_ph, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

# Training
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        for i in range(0, n_samples, batch_size):
            x_batch = X_train[i: i+batch_size]
            y_batch = y_train[i: i+batch_size]
            sess.run(optimizer, feed_dict={X_ph: x_batch, Y_ph: y_batch})

        if epoch % 10 == 0:
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X_ph: X_val, Y_ph: y_val})
            print(f"Epoch {epoch}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}")


    test_acc = sess.run(accuracy, feed_dict={X_ph: X_test, Y_ph: y_test})
    print("test Accuracy: ", test_acc)

Epoch 0, val_loss: 208.2202, val_acc: 0.3333
Epoch 10, val_loss: 1.6998, val_acc: 0.7500
Epoch 20, val_loss: 0.2760, val_acc: 0.8333
Epoch 30, val_loss: 0.1904, val_acc: 0.9167
Epoch 40, val_loss: 0.2080, val_acc: 0.9167
Epoch 50, val_loss: 0.1279, val_acc: 0.9583
Epoch 60, val_loss: 0.1031, val_acc: 0.9167
Epoch 70, val_loss: 0.1216, val_acc: 0.9167
Epoch 80, val_loss: 0.1351, val_acc: 0.9167
Epoch 90, val_loss: 0.1563, val_acc: 0.9167
test Accuracy:  0.93333334


## Problema 4: Crear un modelo para House Prices

In [8]:
# Loading datasets
df = pd.read_csv("data/train.csv")
df = df[["GrLivArea", "YearBuilt", "SalePrice", "YrSold", "LotArea"]].dropna()
#df = df[["GrLivArea", "YearBuilt", "SalePrice", "YrSold", "LotArea", "HouseStyle", "MoSold"]].dropna()
#df = df.dropna()
print(df.shape)
#X = df[["GrLivArea", "YearBuilt",  "YrSold", "LotArea"]].values
X = df[["GrLivArea", "YearBuilt"]].values
y = df[["SalePrice"]].values.reshape(-1, 1)

# Normalizing
X = (X - X.mean(axis=0)) / X.std(axis=0)
y = (y - y.mean()) / y.std()

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# Placerholder
X_ph = tf.placeholder(tf.float32, [None, 2])
Y_ph = tf.placeholder(tf.float32, [None, 1])

# Weights and biases
w1 = tf.Variable(tf.random_normal([2, 64]))
b1 = tf.Variable(tf.zeros([64]))
w2 = tf.Variable(tf.random_normal([64, 1]))
b2 = tf.Variable(tf.zeros([1]))

# Model
l1 = tf.nn.relu(tf.matmul(X_ph, w1) + b1)
out_put = tf.matmul(l1, w2) + b2

#Loss and optimizer
loss = tf.reduce_mean(tf.square(out_put - Y_ph))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

correct_pred = tf.equal(tf.arg_max(out_put, 1), tf.argmax(Y_ph, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(100):
        for i in range(0, len(X_train), batch_size):
            x_batch = X_train[i: i+batch_size]
            y_batch = y_train[i: i+batch_size]
            sess.run(optimizer, feed_dict={X_ph: x_batch, Y_ph: y_batch})

        if epoch % 10 == 0:
            val_loss = sess.run(loss, feed_dict={X_ph: X_val, Y_ph: y_val})
            print(f"Epoch {epoch}, val_loss: {val_loss:4f}")

    test_loss = sess.run(loss, feed_dict={X_ph: X_test, Y_ph: y_test})
    print(f"test_loss: {test_loss:4f}")



(1460, 5)
Epoch 0, val_loss: 0.599012
Epoch 10, val_loss: 0.309312
Epoch 20, val_loss: 0.325310
Epoch 30, val_loss: 0.327118
Epoch 40, val_loss: 0.341841
Epoch 50, val_loss: 0.345836
Epoch 60, val_loss: 0.332679
Epoch 70, val_loss: 0.311613
Epoch 80, val_loss: 0.301660
Epoch 90, val_loss: 0.301401
test_loss: 0.322643


## Problema 5: Crear un modelo para MNIST

In [9]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalizing and reshaping
X_train = X_train.reshape(-1, 784) / 255.0
X_test = X_test.reshape(-1, 784) / 255.0

#X_train = X_train / 255.0
#X_test = X_test / 255.0

# One-hot encode labels
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)
y_train = enc.fit_transform(y_train.reshape(-1, 1))
y_test = enc.transform(y_test.reshape(-1, 1))

# Spliting validation set
X_val = X_train[-5000:]
y_val = y_train[-5000:]
X_train = X_train[:-5000]
y_train = y_train[:-5000]

print(X_train.shape)

# Placeholders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# Model parameters
w1 = tf.Variable(tf.random_normal([784, 128]))
b1 = tf.Variable(tf.zeros([128]))
w2 = tf.Variable(tf.random_normal([128, 10]))
b2 = tf.Variable(tf.zeros([10]))

# Bulding model
l1 = tf.nn.relu(tf.matmul(X, w1) + b1)
logits = tf.matmul(l1, w2) + b2

#Loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

#Accuracy
correct_pred = tf.equal(tf.arg_max(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):
        for i in range(0, len(X_train), 100):
            x_batch = X_train[i: i+100]
            y_batch = y_train[i: i+100]
            sess.run(optimizer, feed_dict={X: x_batch, Y: y_batch})

        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print(f"Epoch {epoch}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}")

    val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_test, Y: y_test})
    print(f"val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}")

(55000, 784)
Epoch 0, val_loss: 0.9046, val_acc: 0.9146
Epoch 1, val_loss: 0.4711, val_acc: 0.9326
Epoch 2, val_loss: 0.3661, val_acc: 0.9404
Epoch 3, val_loss: 0.3247, val_acc: 0.9482
Epoch 4, val_loss: 0.2926, val_acc: 0.9446
Epoch 5, val_loss: 0.2837, val_acc: 0.9490
Epoch 6, val_loss: 0.2746, val_acc: 0.9532
Epoch 7, val_loss: 0.2649, val_acc: 0.9564
Epoch 8, val_loss: 0.2496, val_acc: 0.9600
Epoch 9, val_loss: 0.2762, val_acc: 0.9594
val_loss: 0.2933, val_acc: 0.9535
